# API client for green fashion consumers to know animal ethical information of brands - web scrape data from public websites
website: https://goodonyou.eco/
The function provides a quick look at brandlists animal usage information, standards, and scores.
animal material ethical information include usage of fur, angora, down feather, shearling, karakul, exotic animal skin and hair, wool use including ‘mulesing’, and leather. Also check whether brands use these material align with standards.

skills:
Data Acquisition:
Import data from json.
Web scrape data from public websites.
Write an API client for an API and/or functions associated with API interaction.
Handle, parse, and transform JSON and XML.
Data Cleaning, Transformation, and Organization:
Use data wrangling to transform data into a dataframe ready for green consumers reference and analysis.
Use loops processes.
Use functions and functional programming to export repetitive or difficult tasks.
Handle and process strings and use regular expressions.
Documentation and Presentation:
Provide and document useful functions as part of a Python package.
capture error
Use Jupyter Notebook to generate a vignette on how to use your package functions.

## import package

In [1]:
pip install brandaniethical

Note: you may need to restart the kernel to use updated packages.


In [2]:
from brandaniethical import brandaniethical

# create vignettes - presentation demo (class of popular fashion brand)
I retrieve 6 popoluar footwear and apparel brands' tickers yahoo finance: https://finance.yahoo.com/screener/88f2ce8e-1a19-4f03-ad8f-95e8b2c9c0d1?offset=25&=25
create a class of brandbook for later presentation of features

In [6]:
class trendbrand():
    def __init__(self, name='', btype=''):
        self.name = name
        self.type = btype
    def __str__(self):
        return f"brand name: {self.name}; brand type: {self.type}\n"
    
b1 = trendbrand(name='NIKE', btype = 'footwear')
b2 = trendbrand(name='adidas', btype = 'footwear')
b3 = trendbrand(name='ASICS', btype = 'footwear')
b4 = trendbrand(name='cos', btype = 'apparel')
b5 = trendbrand(name='theory', btype = 'apparel')
b6 = trendbrand(name='lululemon', btype = 'apparel')

class Brandbook:
    def __init__(self):
        self.bb = []

    def add(self, trendbrand):
        self.bb.append(trendbrand)
        
    def __str__(self):
        temp = ""
        for fasbrand in self.bb:
            temp = temp + str(fasbrand.name) + "\n"
        return temp
    
    def get_type(self, brand_type):
        gettype=[]
        for fasbrand in self.bb:
            if brand_type==fasbrand.type:
                gettype.append(fasbrand.name)
        return gettype


#make brand book
bbook = Brandbook()
#add brands
b_list = [b1, b2, b3, b4, b5, b6]
for b in b_list:
    bbook.add(b)

In [7]:
def brandbag(brand_type):  
    return bbook.get_type(brand_type=brand_type)


In [8]:
footwearvig = brandbag('footwear')
apparelvig = brandbag('apparel')


In [4]:
brandaniethical.brandaniethical(['cos','theory','lululemon','nike', 'skechers'])

,company,animal_score,update,fur,angora,leather,wool,shearling,karakul,down,feather,exotic animal hair,exotic animal skin,down_RDS,wool_mulesing
0,cos,3/5,July 2020,0,0,1,1,0,0,1,0,1,0,1,1
1,theory,2/5,December 2019,0,0,1,1,0,0,1,0,1,0,0,0
2,lululemon,2/5,July 2020,0,0,1,1,0,0,1,0,1,0,1,0
3,nike,2/5,July 2020,0,0,1,1,0,0,1,0,1,0,0,0
4,skechers,2/5,August 2020,0,0,1,1,0,0,0,0,0,0,0,0


In [9]:
brandaniethical.brandaniethical(footwearvig)

,company,animal_score,update,fur,angora,leather,wool,shearling,karakul,down,feather,exotic animal hair,exotic animal skin,down_RDS,wool_mulesing
0,nike,2/5,July 2020,0,0,1,1,0,0,1,0,1,0,0,0
1,adidas,3/5,July 2020,0,0,1,1,0,0,0,0,1,0,0,1
2,asics,2/5,August 2020,0,0,1,1,0,0,1,0,1,0,0,1


In [10]:
brandaniethical.brandaniethical(apparelvig)

,company,animal_score,update,fur,angora,leather,wool,shearling,karakul,down,feather,exotic animal hair,exotic animal skin,down_RDS,wool_mulesing
0,cos,3/5,July 2020,0,0,1,1,0,0,1,0,1,0,1,1
1,theory,2/5,December 2019,0,0,1,1,0,0,1,0,1,0,0,0
2,lululemon,2/5,July 2020,0,0,1,1,0,0,1,0,1,0,1,0
